In [1]:
import os
import sys
import pickle
import numpy as np
import torch 
from torch import nn
from tqdm import tqdm

os.environ["CONFIG_PATHS"] = "../configs/small_model.toml,../configs/notebook_overrides.toml"
# os.environ["CONFIG_OVERRIDES"] = "network.policy_head_channels=128"
sys.path.append("../src")

import neural_net
from configuration import config
from neural_net import NeuralNet, ResidualBlock
import player_pov_helpers

""

Loaded config:  {"development": {"debug_mode": true, "profile": false, "runtime": 36000, "display_logs_in_console": false, "output_directory": "data/"}, "game": {"board_size": 10, "num_moves": 6233, "moves_directory": "../data/moves_10/"}, "network": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 64, "residual_blocks": 8}, "architecture": {"gameplay_processes": 6, "coroutines_per_process": 256, "inference_batch_size": 128}, "mcts": {"full_move_probability": 0.2, "full_move_rollouts": 500, "fast_move_rollouts": 100, "ucb_exploration": 1.4, "root_dirichlet_alpha": 0.03, "root_exploration_fraction": 0.25}}
Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners


''

In [2]:
BOARD_SIZE = config()["game"]["board_size"]

In [3]:
def model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum(p.numel() for p in model_parameters)

In [4]:
import time

def time_per_eval(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        occupancies = torch.from_numpy(random_arrays[i]).to(device="mps", dtype=torch.float32)
        model(occupancies)
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [5]:
model = NeuralNet()
model_size(model)
print("Model size:", len(model.residual_blocks))

Model size: 8


In [6]:
time_per_eval(
    num_evaluations=100000000000,
    batch_size=128,
    model=model.to("mps"),
)

Starting...


7.263044621816871e-05

In [7]:
time_per_eval_coreml(
    num_evaluations=10000,
    batch_size=16,
    model=model_from_export,
)

NameError: name 'time_per_eval_coreml' is not defined